# Set up Bedrock client and validate connections

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio on ml.t3.medium instance*

---

In this demo notebook, we demonstrate how to use the boto3 Python SDK to work with Amazon Bedrock Foundation Models. If you are running this in AWS provided accounts, excessive API calls to Bedrock APIs may results in throttling and your account may get blocked

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. 
#### ⚠️ You will see pip dependency errors, broken permissions you can safely ignore these errors. ⚠️

In [ ]:
# let's upgrade python package manager to latest version
%pip install --upgrade pip


In [ ]:

%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install other packages. Below are some of the Python Packages used in subsuquent labs
#### - Langchain - LangChain is a framework for developing applications powered by language models
#### - pypdf - Python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files
#### - finance - yfinance offers a threaded and Pythonic way to download market data from Yahoo
#### - Pandas-datareader - used to access public financial data and import it into Python as a DataFrame
#### - sqllite3 - lightweight disk-based database that doesn't require a separate server process
#### - duckduckgo-search - light weight search engine

### text

In [ ]:

%pip install --quiet \
    "langchain==0.1.14" \
    sqlalchemy -U \
    "pypdf>=3.8,<4" \
    datasets \
    "ipywidgets>=7,<8" \
    matplotlib \
    pymupdf \
    rich 


### agents

In [ ]:

%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    pysqlite3 \
    google-search-results


### entity extraction

In [ ]:

#Beautiful Soup is a Python library for pulling data out of HTML and XML files
%pip install --quiet beautifulsoup4


### Restart Kernel 

In [ ]:

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")



---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

Depending on your environment, you might need to customize the setup when creating your Bedrock service client. To help with this, we've provided a `get_bedrock_client()` utility method that supports passing in different options. You can find the implementation in [../utils/bedrock.py](../utils/bedrock.py)

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).

The `get_bedrock_client()` method accepts `runtime` (default=True) parameter to return either `bedrock` or `bedrock-runtime` client.

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### A note about `langchain`

The Bedrock classes provided by `langchain` create a Bedrock boto3 client by default. To customize your Bedrock configuration, we recommend to explicitly create the Bedrock client using the method below, and pass it to the [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) class instantiation method using `client=boto3_bedrock`

In [ ]:

import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)


#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use 

In [ ]:

boto3_bedrock.list_foundation_models()


## Congrats !

You have successfully installed required Python SDK and other packages needed to run other notebooks in the bootcamp. Please proceed to the next notebooks 